In [1]:
#LM #LANGUAGE MODEL STUFFS
#http://abay.tech/blog/2018/04/11/how-to-use-googles-pre-trained-language-model/
from lm1b import LoadModel
# From lm_1b
import language_model.lm_1b.data_utils as data_utils
import pickle as pkl
from six.moves       import xrange
from google.protobuf import text_format
import numpy as np
from tqdm import tqdm

/usr/local/lib/python2.7/dist-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:169: R

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [3]:
# File Paths
vocab_file = "language_model/data/vocab-2016-09-10.txt"
save_dir   = "language_model/output"
pbtxt      = "language_model/data/graph-2016-09-10.pbtxt"
ckpt       = "language_model/data/ckpt-*"

sess2, t,tgd = LoadModel(pbtxt, ckpt)

# For saving demo resources, use batch size 1 and step 1.
BATCH_SIZE = 1
NUM_TIMESTEPS = 1
MAX_WORD_LEN = 50

#Vocabulary containing character-level information.
vocab = data_utils.CharsVocabulary(vocab_file, MAX_WORD_LEN)

targets  = np.zeros([BATCH_SIZE, NUM_TIMESTEPS], np.int32)
weights  = np.ones([BATCH_SIZE, NUM_TIMESTEPS], np.float32)
inputs   = np.zeros([BATCH_SIZE, NUM_TIMESTEPS], np.int32)
char_ids_inputs = np.zeros([BATCH_SIZE, NUM_TIMESTEPS, vocab.max_word_length], np.int32)

# Recovers the model from protobuf


INFO:tensorflow:Recovering Graph language_model/data/graph-2016-09-10.pbtxt


In [4]:


def forward(sentence):
    sess2.run(t['states_init'])
    
    if sentence.find('<S>') != 0:
        sentence = '<S> ' + sentence

    char_ids = [vocab.word_to_char_ids(w) for w in sentence.split()]
    #print len(char_ids)
    for i in xrange(len(char_ids)):
        char_ids_inputs[0, 0, :] = char_ids[i]
        lstm_emb = sess2.run(t['lstm/lstm_1/control_dependency'],
                        feed_dict={t['char_inputs_in']: char_ids_inputs,
                                  })
    return lstm_emb

In [5]:
def getSenTargetBatch(batchSample):
    senEmbSet=np.array([forward(' '.join(i[1])) for i in tqdm(batchSample)])
    return senEmbSet

In [6]:
with open('./captions/token_train.pkl') as f:
    trainCaptions=pkl.load(f)


In [7]:
yp=getSenTargetBatch(trainCaptions)

100%|██████████| 49246/49246 [9:50:23<00:00,  1.39it/s]  
